In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [1]:

import time
from PyPDF2 import PdfReader
from PIL import Image

import os


# from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
import openai
from langchain import PromptTemplate


In [2]:
import io
import requests
from PyPDF2 import PdfReader
headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}

url = 'https://services.google.com/fh/files/misc/practitioners_guide_to_mlops_whitepaper.pdf'
response = requests.get(url=url, headers=headers, timeout=120)
on_fly_mem_obj = io.BytesIO(response.content)
pdf_file = PdfReader(on_fly_mem_obj)

In [7]:
openai_api_key = os.environ.get("OPENAI_API_KEY",None)

In [8]:
text = ""
for page in pdf_file.pages:
    text += page.extract_text()

In [22]:
text_splitter = CharacterTextSplitter(
separator="\n",
chunk_size=1000,
chunk_overlap=200,
length_function=len
)

chunks = text_splitter.split_text(text)
    
# create docs
docs = [Document(page_content=t) for t in chunks[:10]]
llm = OpenAI(temperature=0, openai_api_key=openai_api_key)

# show summarize doc
chain = load_summarize_chain(llm, chain_type="map_reduce")
summarized_docs = chain.run(docs)
print("Summary")
print(summarized_docs)


Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised Timeout: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600).


Summary
 This white paper provides a framework for MLOps, a continuous delivery and automation process for machine learning. It covers topics such as ML development, training operationalization, continuous training, model deployment, prediction serving, continuous monitoring, and data and model management. It outlines an MLOps framework that defines core processes and technical capabilities to help organizations establish mature MLOps practices for building and operationalizing ML systems. It also discusses the challenges organizations face when attempting to deploy ML at scale, such as lack of talent, integration issues, and deployment challenges. ML engineering is essential for building ML-enabled systems, and MLOps is a superset of software engineering that deals with the complexities of practical ML applications.


In [23]:
print(summarized_docs,sep='\n')

 This white paper provides a framework for MLOps, a continuous delivery and automation process for machine learning. It covers topics such as ML development, training operationalization, continuous training, model deployment, prediction serving, continuous monitoring, and data and model management. It outlines an MLOps framework that defines core processes and technical capabilities to help organizations establish mature MLOps practices for building and operationalizing ML systems. It also discusses the challenges organizations face when attempting to deploy ML at scale, such as lack of talent, integration issues, and deployment challenges. ML engineering is essential for building ML-enabled systems, and MLOps is a superset of software engineering that deals with the complexities of practical ML applications.


In [24]:
def insert_line_break(text, words_per_line=10):
    words = text.split()
    formatted_text = "\n".join(" ".join(words[i:i+words_per_line]) for i in range(0, len(words), words_per_line))
    return formatted_text

In [25]:
formatted_text = insert_line_break(summarized_docs)

# Display the formatted text as Markdown
Markdown(formatted_text)

This white paper provides a framework for MLOps, a continuous
delivery and automation process for machine learning. It covers topics
such as ML development, training operationalization, continuous training, model deployment,
prediction serving, continuous monitoring, and data and model management. It
outlines an MLOps framework that defines core processes and technical
capabilities to help organizations establish mature MLOps practices for building
and operationalizing ML systems. It also discusses the challenges organizations
face when attempting to deploy ML at scale, such as
lack of talent, integration issues, and deployment challenges. ML engineering
is essential for building ML-enabled systems, and MLOps is a
superset of software engineering that deals with the complexities of
practical ML applications.